In [ ]:
import SIRmodels as mdl
import analysisSI1I2R as an
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
#%matplotlib qt
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['lines.linewidth'] = 1
matplotlib.rcParams['figure.titlesize'] = 18
matplotlib.rcParams['axes.labelsize'] = 16
matplotlib.rcParams['xtick.labelsize'] = 14
matplotlib.rcParams['ytick.labelsize'] = 14
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['legend.fontsize'] = 14

# SIR interactive model 

## Change B' and D'

In [ ]:
# Originals values
N = 100000
SIIR0 = np.zeros(9)
SIIR0[1] = 10
SIIR0[2] = 10
SIIR0[0] = N - np.sum(SIIR0[1:8])

t_start = 0
t_end = 22
n_int = 10000

t_sim = np.linspace(t_start, t_end, n_int)

In [ ]:
#params = [6.15, 0, 5.5, 0, 6.15, 0, 5.5/30, 0]  # Big beta1' generates synch
#params = [6.15, 0, 5.5, 0, 6.15, 0, 5.5/0.1, 0]  # Big delta1' generates asynch
params = [6.15, 0, 5.5, 0, 6.15, 0, 5.5, 0]  # Independent case


#params = [6.15, 0, 5.5, 0, 6.15, 0, 0.3, 0]  # Big beta1' generates synch
#params = [6.15, 0, 5.5, 0, 6.15, 0,19.5, 0]  # Big delta1' generates asynch


fig, ax = plt.subplots(2, 2)

params_copy = params.copy()
beta_prime_arr = np.array([4, 5, 6.15, 10, 15])
delta_prime_arr = beta_prime_arr/1.12
f_size1 = []
f_size2 = []
for i in range(len(beta_prime_arr)):
    params_copy[1] = beta_prime_arr[i]
    params_copy[5] = beta_prime_arr[i]
    params_copy[3] = delta_prime_arr[i]
    params_copy[7] = delta_prime_arr[i]
    siirSim = mdl.SIIR(SIIR0, params_copy, t_sim)
    siirSim.runEvaluation(norm=True)
    res = siirSim.getResult()
    
    inf1 = res[:, 1] + res[:, 3] + res[:, 7]
    rec1 = res[:, 4] + res[:, 6] + res[:, 8]
    f_size1.append(rec1[-1])
    ax[0,0].plot(t_sim, inf1, label=r'$\beta_2$=' + str(round(beta_prime_arr[i],2)) 
                                     + r', $\delta_2$=' + str(round(delta_prime_arr[i],2)))
    ax[0,1].plot(t_sim, rec1)
    
    
    inf2 = res[:, 2] + res[:, 3] + res[:, 6]
    rec2 = res[:, 5] + res[:, 7] + res[:, 8]
    f_size2.append(rec2[-1])
    ax[1,0].plot(t_sim, inf2)
    ax[1,1].plot(t_sim, rec2)

#ax[0,0].legend()
ax[0,0].set_title(r"(a) $P_{I_{1}}$")
ax[0,0].set_xlabel(r"time")
ax[0,0].set_ylabel(r"$P_{I_1}$")
#ax[0,1].legend()
ax[0,1].set_title(r"(b) $P_{R_{1}}$")
ax[0,1].set_xlabel(r"time")
ax[0,1].set_ylabel(r"$P_{R_{1}}$")
#ax[1,0].legend()
ax[1,0].set_title(r"(c) $P_{I_{2}}$")
ax[1,0].set_xlabel(r"time")
ax[1,0].set_ylabel(r"$P_{I_2}$")
#ax[1,1].legend()
ax[1,1].set_title(r"(d) $P_{R_{2}}$")
ax[1,1].set_xlabel(r"time")
ax[1,1].set_ylabel(r"$P_{R_2}$")
fig.legend(loc=(0.8,0.6),fontsize=16)
fig.tight_layout(h_pad=-2, w_pad=-1.5)
fig.set_size_inches(15, 7.5)
nameFile = '1-3.svg'
plt.savefig('images/summary2/'+nameFile)

## 2D plots

#### Beta2 vs Delta1'

In [ ]:
params = [6.15, 0, 5.5, 0, 6.15, 0, 5.5, 0]
beta2_arr = np.arange(0,20.1,0.1)
delta1prime_arr = np.arange(0,20.1,0.1)
rate = 1.12
params_copy = params.copy()
Delta1prime, Beta2 = np.meshgrid(delta1prime_arr,beta2_arr)

peak1 = np.zeros((len(beta2_arr), len(delta1prime_arr)))
peak2 = np.zeros((len(beta2_arr), len(delta1prime_arr)))
inf1 = np.zeros((len(beta2_arr), len(delta1prime_arr)))
inf2 = np.zeros((len(beta2_arr), len(delta1prime_arr)))
for j in np.arange(len(delta1prime_arr)):
    delta1prime = delta1prime_arr[j]
    params_copy[6] = delta1prime
    for i in np.arange(len(beta2_arr)):
        beta2 = beta2_arr[i]
        print("(beta2,delta1prime)", beta2, delta1prime)
        params_copy[1] = beta2
        params_copy[3] = beta2/rate
        params_copy[5] = beta2
        params_copy[7] = beta2/rate
        siir = mdl.SIIR(SIIR0, params_copy, t_sim)
        siir.runEvaluation(norm=True)
        #print(t_sim[siir.getDisease2()[1]])
        peak1[i, j] =  0 if siir.getDisease1()[1].size==0 else t_sim[siir.getDisease1()[1]][0] # getPeak1
        peak2[i, j] = 0 if siir.getDisease2()[1].size==0 else t_sim[siir.getDisease2()[1]][0]
        inf1[i, j] = siir.getNInfected1() # n infected 1
        inf2[i, j] = siir.getNInfected2() # n infected 2
        
import pickle as pkl

nameFile = "synch_" 
pkl_file = open(nameFile + 'peak1.pkl', 'wb')
pkl.dump(peak1,pkl_file)
pkl_file.close()
pkl_file = open(nameFile + 'peak2.pkl', 'wb')
pkl.dump(peak2,pkl_file)
pkl_file.close()
pkl_file = open(nameFile + 'inf1.pkl', 'wb')
pkl.dump(inf1,pkl_file)
pkl_file.close()
pkl_file = open(nameFile + 'inf2.pkl', 'wb')
pkl.dump(inf2,pkl_file)
pkl_file.close()

In [ ]:
peak1 = pkl.load(open("synch_peak1.pkl",'rb'))
peak2 = pkl.load(open("synch_peak2.pkl",'rb'))
# Plot: Peak
fig, ax = plt.subplots(1, 1)
ax.set_xlabel(r"$\delta_1'$",fontsize=20)
ax.set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr)+0.001)
ax.set_ylabel(r"$\beta_2$",fontsize=20)
ax.set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax.set_title(r"$\tau_1$",fontsize=24)
ax.set_aspect('equal')
ax.plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r',linewidth=2.5,
       label=r"$\delta_1'=\delta_1$")  
ax.legend(fontsize=18)
ax.set_xticks(np.arange(0,20.0001,5))
ax.set_yticks(np.arange(0,20.0001,5))
from scipy.ndimage.filters import gaussian_filter
peak1 = gaussian_filter(peak1, 1.5)
ax.contour(Delta1prime, Beta2, peak1,5,colors='k')
cf1 = ax.contourf( Delta1prime,Beta2, peak1, np.arange(8.12,8.68,0.001))
#ax[0].contour(Beta1prime, Beta2prime, peak1, 10, colors='black')
cb1 = fig.colorbar(cf1, ax=ax)
cb1.set_ticks(np.linspace(8.120,8.68,5))
cb1.set_label(r"$\tau_1$",fontsize=20)
for c in cf1.collections:
    c.set_edgecolor("face")
ax.plot([0.3],[5.1],'or',[params[2]],[5.1],'xw',[19.5],[5.1],'sy', ms=10)


fig.set_size_inches(8,6)
nameFile = '2-1.svg'
plt.savefig('images/summary2/'+nameFile)

In [ ]:
inf1 = pkl.load(open("synch_inf1.pkl",'rb'))
inf2 = pkl.load(open("synch_inf2.pkl",'rb'))
# Plot: infected
fig, ax = plt.subplots(1, 1)
ax.set_xlabel(r"$\delta_1'$",fontsize=20)
ax.set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr))
ax.set_ylabel(r"$\beta_2$",fontsize=20)
ax.set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax.set_title(r"$P_{R_{1}\infty}$",fontsize=20)
ax.set_aspect('equal')
ax.plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r',linewidth=2.5,
       label=r"$\delta_1'=\delta_1$")
ax.legend(fontsize=18)
ax.set_xticks(np.arange(0,20.0001,5))
ax.set_yticks(np.arange(0,20.0001,5))
#ax[0].legend(('Beta1', 'Beta2'), fontsize=fontsize)
#ax.legend(r"$\delta_1'=\delta_1$", fontsize=18)
ax.contour(Delta1prime, Beta2, inf1,5,colors='k')
cf1 = ax.contourf( Delta1prime,Beta2, inf1, np.arange(0.196,0.2265,0.0001))
#ax[0].contour(Beta1prime, Beta2prime, peak1, 10, colors='black')
cb1 = fig.colorbar(cf1, ax=ax)
cb1.set_label(r"$P_{R_{1}\infty}$")
cb1.set_ticks(np.linspace(0.196,0.226,5))
for c in cf1.collections:
    c.set_edgecolor("face")
ax.plot([0.3],[5.1],'or',[params[2]],[5.1],'xw',[19.5],[5.1],'sy', ms=10)
ax.plot([0.3],[5.1],'or',[params[2]],[5.1],'xw',[19.5],[5.1],'sy',ms=10)


#cb2 = fig.colorbar(cf2, ax=ax2)

fig.set_size_inches(8, 6)
nameFile = '2-2.svg'
plt.savefig('images/summary2/'+nameFile)

In [ ]:
# Copy cell Peak
'''# Plot: Peak
fontsize = 8
fig, ax = plt.subplots(1, 2)
ax[0].set_xlabel(r"$\delta_1'$")
ax[0].set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr))
ax[0].set_ylabel(r"$\beta_2$")
ax[0].set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax[0].set_title('Disease 1')
ax[0].set_aspect('equal')
ax[0].plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r')  
ax[0].plot(beta2_arr, params[0] * np.ones(len(beta2_arr)), '-r') 
#ax[0].legend(('Beta1', 'Beta2'), fontsize=fontsize)
cf1 = ax[0].contourf( Delta1prime,Beta2, peak1, 30, cmap='gist_yarg')
#ax[0].contour(Beta1prime, Beta2prime, peak1, 10, colors='black')
cb1 = fig.colorbar(cf1, ax=ax[0])
cb1.set_label('Peak position')
txt = ("Beta1 = " + str(params[0]) + "\n Delta1 = " + str(params[2])
       + "\n Beta1\' = " + str(params[4]) + "\n Delta1\' = " + str(params[6]))
props = dict(boxstyle='square', facecolor='wheat', alpha=0.8)
ax[0].text(0.6, 0.4, txt, ha='left', transform=ax[0].transAxes, fontsize=fontsize, bbox=props, wrap=True)        

ax[1].set_xlabel(r"$\delta_1'$")
ax[1].set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr))
ax[1].set_ylabel(r"$\beta_2$")
ax[1].set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax[1].set_title('Disease 2')
ax[1].set_aspect('equal')
ax[1].plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r')  
ax[1].plot(beta2_arr, params[0] * np.ones(len(beta2_arr)), '-r') 
#ax[1].legend(('Beta1', 'Beta2'), fontsize=fontsize)
cf2 = ax[1].contourf( Delta1prime,Beta2, peak2, 30, cmap='gist_yarg')
#ax[1].contour(Beta1prime, Beta2prime, peak2, 10, colors='black')
cb2 = fig.colorbar(cf2, ax=ax[1])
cb2.set_label('Peak position')
txt = ("Beta2 = " + str(params[0]) + "\n Delta2 = " + str(params[2])
       + "\n Beta2\' = " + str(params[4]) + "\n Delta2\' = " + str(params[6]))
props = dict(boxstyle='square', facecolor='wheat', alpha=0.8)
ax[1].text(0.6, 0.4, txt, ha='left', transform=ax[1].transAxes, fontsize=fontsize, bbox=props, wrap=True)
fig.set_size_inches(12, 6)
nameFile = '2-1.svg'
plt.savefig('images/summary2/'+nameFile)'''

In [ ]:
# Plot: infected
fontsize = 8
fig, ax = plt.subplots(1, 2)
ax[0].set_xlabel('Delta1\'')
ax[0].set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr))
ax[0].set_ylabel('Beta2')
ax[0].set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax[0].set_title('Disease 1')
ax[0].set_aspect('equal')
ax[0].plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r')  
ax[0].plot(beta2_arr, params[0] * np.ones(len(beta2_arr)), '-r') 
#ax[0].legend(('Beta1', 'Beta2'), fontsize=fontsize)
cf1 = ax[0].contourf( Delta1prime,Beta2, inf1, 30, cmap='gist_yarg')
#ax[0].contour(Beta1prime, Beta2prime, peak1, 10, colors='black')
cb1 = fig.colorbar(cf1, ax=ax[0])
cb1.set_label('Peak position')
txt = ("Beta1 = " + str(params[0]) + "\n Delta1 = " + str(params[2])
       + "\n Beta1\' = " + str(params[4]) + "\n Delta1\' = " + str(params[6]))
props = dict(boxstyle='square', facecolor='wheat', alpha=0.8)
ax[0].text(0.6, 0.4, txt, ha='left', transform=ax[0].transAxes, fontsize=fontsize, bbox=props, wrap=True)        

ax[1].set_xlabel('Delta1\'')
ax[1].set_xlim(np.min(delta1prime_arr),np.max(delta1prime_arr))
ax[1].set_ylabel('Beta2')
ax[1].set_ylim(np.min(beta2_arr),np.max(beta2_arr))
ax[1].set_title('Disease 2')
ax[1].set_aspect('equal')
ax[1].plot(params[6] * np.ones(len(delta1prime_arr)), delta1prime_arr, '-.r')  
ax[1].plot(beta2_arr, params[0] * np.ones(len(beta2_arr)), '-r') 
#ax[1].legend(('Beta1', 'Beta2'), fontsize=fontsize)
cf2 = ax[1].contourf( Delta1prime,Beta2, inf2, 30, cmap='gist_yarg')
#ax[1].contour(Beta1prime, Beta2prime, peak2, 10, colors='black')
cb2 = fig.colorbar(cf2, ax=ax[1])
cb2.set_label('Peak position')
txt = ("Beta2 = " + str(params[0]) + "\n Delta2 = " + str(params[2])
       + "\n Beta2\' = " + str(params[4]) + "\n Delta2\' = " + str(params[6]))
props = dict(boxstyle='square', facecolor='wheat', alpha=0.8)
ax[1].text(0.6, 0.4, txt, ha='left', transform=ax[1].transAxes, fontsize=fontsize, bbox=props, wrap=True)
nameFile = '2-2.svg'
plt.savefig('images/summary2/'+nameFile)